In [68]:
import os
import pathlib
import osmnx as ox
import pickle
import candidates_preparation
import IVMM
import pandas as pd
from utils.vector_haversine_distances import haversine_np

In [33]:
%%time
# fetch road graph from openstreetmap using osmnx
# and save it to local file
# road_network = ox.graph_from_place("Shanghai, China", network_type='drive', which_result=2)
# ox.io.save_graphml(city, filepath='db/shanghai-drive-20200902.gml', gephi=False, encoding='utf-8')
# ox.plot_graph(road_network)
# load shanghai road network graphml file
road_network = ox.load_graphml('db/shanghai-drive-20200902.gml')

edges = ox.utils_graph.graph_to_gdfs(road_network, nodes=False, fill_edge_geometry=True)
edges.drop(columns=['osmid', 'highway', 'bridge', 'name', 'key'], inplace=True)

# extended = candidates_preparation.extended_edges(edges)
# with open('db/shenzhen_drive_edges_extend.pkl', 'wb') as f:
#     pickle.dump(extended, f)
## extended edges is created by above code segment
with open('db/shanghai_drive_edges_extend.pkl', 'rb') as f:
    extended = pickle.load(f)

# tree = candidates_preparation.make_tree(extended)
# with open('db/shenzhen_drive_extended_edges_tree.pkl', 'wb') as f:
#     pickle.dump(tree, f)
## tree is created by above code segment
with open('db/shanghai_drive_extended_edges_tree.pkl', 'rb') as f:
    tree = pickle.load(f)

# The data_list is all files to be processed
db_path = 'db/shanghai/Taxi_070220'
data_list = sorted(os.listdir(db_path))

pathlib.Path('result/shanghai/road_path').mkdir(parents=True, exist_ok=True)

Wall time: 37.5 s


In [49]:
# for i th trajectory file
file_path = os.path.join(db_path, data_list[0])
trajectory = pd.read_csv(file_path, header=None, names=['plate', 'timestamp', 'longitude', 'latitude', 'speed', 'direction', 'status'], parse_dates=['timestamp'])
trajectory['dis_f_pre'] = haversine_np(trajectory['longitude'], trajectory['latitude'], trajectory['longitude'].shift(), trajectory['latitude'].shift())

In [70]:
%%time
# processs a task
IVMM.IVMM(trajectory, road_network, edges, extended, tree, res_path='result/shanghai/road_path', seg_name=data_list[0])

Wall time: 1min 51s
